# networking

In [ ]:
open rust.rust_operators

In [ ]:
//// test

open testing

## rust

### reqwest_response

In [ ]:
nominal reqwest_response =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"reqwest_wasm::Response\")>]\n#endif\ntype reqwest_Response = class end"
        $'' : $'reqwest_Response'
    )

### reqwest_error

In [ ]:
nominal reqwest_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"reqwest_wasm::Error\")>]\n#endif\ntype reqwest_Error = class end"
        $'' : $'reqwest_Error'
    )

### request_builder

In [ ]:
nominal request_builder =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"reqwest_wasm::RequestBuilder\")>]\n#endif\ntype reqwest_RequestBuilder = class end"
        $'' : $'reqwest_RequestBuilder'
    )

### request_type

In [ ]:
union request_type =
    | Get
    | Post

### request

In [ ]:
type request =
    {
        url : string
        request_type : request_type
        body : string
        json : bool
        auto_refresh : bool
    }

### new_request_get

In [ ]:
inl new_request_get (url : string) : request_builder =
    inl url = join url
    inl url = url |> sm'.to_std_string
    inl url = join url
    !\($'"reqwest_wasm::Client::builder().build().map_err(|err| err.to_string())?.get(!url)"')

### new_request_post

In [ ]:
inl new_request_post (url : string) : request_builder =
    inl url = join url
    inl url = url |> sm'.to_std_string
    inl url = join url
    !\($'"reqwest_wasm::Client::builder().build().map_err(|err| err.to_string())?.post(!url)"')

### request_send

In [ ]:
inl request_send (request : request_builder) : async.future_pin (resultm.result' reqwest_response reqwest_error) =
    inl request = join request
    !\($'"Box::pin(reqwest_wasm::RequestBuilder::send(!request))"')

### request_body

In [ ]:
inl request_body (body : string) (request : request_builder) : request_builder =
    inl body = body |> sm'.to_std_string
    !\\(body, $'"reqwest_wasm::RequestBuilder::body(!request, $0)"')

### request_header

In [ ]:
inl request_header (key : string) (value : string) (request : request_builder) : request_builder =
    inl request = join request
    inl key = key |> sm'.to_std_string
    inl value = value |> sm'.to_std_string
    !\\((key, value), $'"reqwest_wasm::RequestBuilder::header(!request, $0, $1)"')

### request_json

In [ ]:
inl request_json forall t. (obj : t) (request : request_builder) : request_builder =
    !\($'"reqwest_wasm::RequestBuilder::json(!request, &!obj)"')

### response_text

In [ ]:
inl response_text (response : reqwest_response) : async.future_pin (resultm.result' sm'.std_string reqwest_error) =
    !\($'"Box::pin(reqwest_wasm::Response::text(!response))"')

## fsharp

### tcp_client

In [ ]:
nominal tcp_client =
    `(
        global "#if FABLE_COMPILER\n\ntype System_Net_Sockets_TcpClient = System.IDisposable\n#else\ntype System_Net_Sockets_TcpClient = System.Net.Sockets.TcpClient\n#endif\n"
        $'' : $'System_Net_Sockets_TcpClient'
    )

### new_tcp_client

In [ ]:
inl new_tcp_client () : tcp_client =
    run_target function
        | Fsharp (Native) => fun () => $'new `tcp_client ()'
        | _ => fun () => null ()

### ip_address

In [ ]:
nominal ip_address = $'System.Net.IPAddress'

### ip_address_parse

In [ ]:
inl ip_address_parse (s : string) : ip_address =
    s |> $'`ip_address.Parse'

### tcp_listener

In [ ]:
nominal tcp_listener = $'System.Net.Sockets.TcpListener'

### new_tcp_listener

In [ ]:
inl new_tcp_listener (ip_address : ip_address) (port : i32) : tcp_listener =
    $'new `tcp_listener (!ip_address, !port)'

### listener_start

In [ ]:
inl listener_start (listener : tcp_listener) : () =
    listener |> $'_.Start()'

### listener_stop

In [ ]:
inl listener_stop (listener : tcp_listener) : () =
    listener |> $'_.Stop()'

### client_connect_async

In [ ]:
inl client_connect_async
    (host : string)
    (port : i32)
    (ct : threading.cancellation_token)
    (client : tcp_client)
    : async.value_task
    =
    run_target function
        | Fsharp (Native) => fun () => $'!client.ConnectAsync (!host, !port, !ct)'
        | _ => fun () => null ()

### test_port_open

In [ ]:
let test_port_open host port : _ bool = async.new_async fun () =>
    inl ct = async.cancellation_token () |> async.let'
    inl client = new_tcp_client () |> use
    try_unit
        fun () =>
            client |> client_connect_async host port ct |> async.await_value_task_unit |> async.do
            return true
        fun ex =>
            trace Verbose
                fun () => "networking.test_port_open"
                fun () => { port ex = ex () |> sm'.format_exception }
            return false

In [ ]:
//// test

test_port_open "127.0.0.1" 65536
|> async.run_with_timeout 120
|> _assert_eq (Some false)

00:00:00 v #1 networking.test_port_open / { port = 65536; ex = System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'port') }
__assert_eq / actual: US6_0 false / expected: US6_0 false


### test_port_open_timeout

In [ ]:
let test_port_open_timeout timeout host port : _ bool = async.new_async_unit fun () =>
    test_port_open host port
    |> async.run_with_timeout_async timeout
    |> async.let'
    |> function
        | None => false
        | Some result => result
    |> return

In [ ]:
//// test

test_port_open_timeout 120 "127.0.0.1" 65535
|> async.run_synchronously
|> _assert_eq false

00:00:00 v #1 async.run_with_timeout_async / { timeout = 120 }
__assert_eq / actual: false / expected: false


### wait_for_port_access

In [ ]:
let wait_for_port_access timeout status host port : _ i64 =
    let rec loop retry : _ i64 =
        fun () =>
            inl is_port_open =
                match timeout |> optionm'.unbox with
                | None => test_port_open host port
                | Some timeout => test_port_open_timeout timeout host port
                |> async.let'
            fix_condition
                fun () => is_port_open = status
                fun () => retry |> return
                fun () =>
                    if retry % 100 = 0 then
                        trace Verbose
                            fun () => "networking.wait_for_port_access"
                            fun () => { port retry timeout status }
                    async.sleep 10 |> async.do
                    loop (retry + 1) |> async.return_await
        |> async.new_async_unit
    loop 1i64

In [ ]:
//// test

inl lock_port host port = async.new_async fun () =>
    trace Debug (fun () => "_1") id
    async.sleep 5000 |> async.do
    inl listener = new_tcp_listener (host |> ip_address_parse) port |> use
    trace Debug (fun () => "_2") id
    listener |> listener_start
    trace Debug (fun () => "_3") id
    async.sleep 2000 |> async.do
    trace Debug (fun () => "_4") id
    $'!listener.Stop' ()
    trace Debug (fun () => "_5") id

inl host = "127.0.0.1"
inl port = 5555i32

fun () =>
    trace Debug (fun () => "1") id
    inl child = lock_port host port |> async.start_child |> async.let'
    trace Debug (fun () => "2") id
    async.sleep 1 |> async.do
    trace Debug (fun () => "3") id
    inl retries1 = wait_for_port_access (None |> optionm'.box) true host port |> async.let'
    trace Debug (fun () => "4") id
    inl retries2 = wait_for_port_access (None |> optionm'.box) false host port |> async.let'
    trace Debug (fun () => "5") id
    child |> async.do
    trace Debug (fun () => "6") id
    (retries1, retries2) |> return
|> async.new_async_unit
|> async.run_with_timeout 20000
|> function
    | Some (retries1, retries2) =>
        retries1
        |> _assert_between
            if platform.is_windows () then 2i64 else 2
            if platform.is_windows () then 5 else 1500

        retries2
        |> _assert_between
            if platform.is_windows () then 80i64 else 80
            if platform.is_windows () then 200 else 600

        true
    | _ => false
|> _assert_eq true

00:00:00 d #1 1
00:00:00 d #2 _1
00:00:00 d #3 2
00:00:00 d #4 3
00:00:02 v #5 networking.test_port_open / { port = 5555; ex = System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.) }
00:00:04 v #6 networking.test_port_open / { port = 5555; ex = System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.) }
00:00:05 d #7 _2
00:00:05 d #8 _3
00:00:05 d #9 4
00:00:06 v #10 networking.wait_for_port_access / { port = 5555; retry = 100; timeout = None; status = false }
00:00:07 d #11 _4
00:00:07 d #12 _5
00:00:09 v #13 networking.test_port_open / { port = 5555; ex = System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.) }
00:00:09 d #14 5
00:00:09 d #15 6
__assert_between / actual: 3L / expected: struct (2L, 5L)
__assert_between / actual: 119L / expected: struct (

In [ ]:
//// test

inl lock_port host port = async.new_async_unit fun () =>
    trace Debug (fun () => "_1") id
    async.sleep 500 |> async.do
    inl listener = new_tcp_listener (ip_address_parse host) port |> use
    trace Debug (fun () => "_2") id
    listener |> listener_start
    trace Debug (fun () => "_3") id
    async.sleep 200 |> async.do
    trace Debug (fun () => "_4") id
    listener |> listener_stop
    trace Debug (fun () => "_5") id

inl host = "127.0.0.1"
inl port = 5555

fun () =>
    trace Debug (fun () => "1") id
    inl child = lock_port host port |> async.start_child |> async.let'
    trace Debug (fun () => "2") id
    async.sleep 1 |> async.do
    trace Debug (fun () => "3") id
    inl retries1 = wait_for_port_access (Some 60 |> optionm'.box) true host port |> async.let'
    trace Debug (fun () => "4") id
    inl retries2 = wait_for_port_access (Some 60 |> optionm'.box) false host port |> async.let'
    trace Debug (fun () => "5") id
    child |> async.do
    trace Debug (fun () => "6") id
    (retries1, retries2) |> return
|> async.new_async_unit
|> async.run_with_timeout 2000
|> function
    | Some (retries1, retries2) =>
        retries1
        |> _assert_between
            if platform.is_windows () then 4i64 else 2
            if platform.is_windows () then 15 else 150

        retries2
        |> _assert_between
            if platform.is_windows () then 5i64 else 0
            if platform.is_windows () then 20 else 60

        true
    | _ => false
|> _assert_eq true

00:00:00 d #1 1
00:00:00 d #2 2
00:00:00 d #3 _1
00:00:00 d #4 3
00:00:00 v #5 async.run_with_timeout_async / { timeout = 60 }
00:00:00 v #6 async.run_with_timeout_async / { timeout = 60 }
00:00:00 v #7 async.run_with_timeout_async / { timeout = 60 }
00:00:00 v #8 async.run_with_timeout_async / { timeout = 60 }
00:00:00 v #9 async.run_with_timeout_async / { timeout = 60 }
00:00:00 d #10 _2
00:00:00 v #11 async.run_with_timeout_async / { timeout = 60 }
00:00:00 d #12 _3
00:00:00 d #13 4
00:00:00 d #14 _4
00:00:00 d #15 _5
00:00:00 v #16 async.run_with_timeout_async / { timeout = 60 }
00:00:00 d #17 5
00:00:00 d #18 6
__assert_between / actual: 7L / expected: struct (4L, 15L)
__assert_between / actual: 14L / expected: struct (5L, 20L)
__assert_eq / actual: true / expected: true


### get_available_port

In [ ]:
let get_available_port timeout host initial_port : _ i32 =
    let rec loop port =
        fun () =>
            inl is_port_open =
                match timeout |> optionm'.unbox with
                | None => test_port_open host port
                | Some timeout => test_port_open_timeout timeout host port
                |> async.let'
            fix_condition
                fun () => is_port_open |> not
                fun () => port |> return
                fun () => loop (port + 1) |> async.return_await
        |> async.new_async_unit
    loop initial_port

In [ ]:
//// test

inl lock_ports host port = async.new_async_unit fun () =>
    trace Debug (fun () => "_1") id
    inl listener1 = new_tcp_listener (ip_address_parse host) port |> use
    inl listener2 = new_tcp_listener (ip_address_parse host) (port + 1) |> use
    trace Debug (fun () => "_2") id
    listener1 |> listener_start
    listener2 |> listener_start
    trace Debug (fun () => "_3") id
    async.sleep 4000 |> async.do
    trace Debug (fun () => "_4") id
    listener1 |> listener_stop
    listener2 |> listener_stop
    trace Debug (fun () => "_5") id

inl host = "127.0.0.1"
inl port = 5555

fun () =>
    trace Debug (fun () => "1") id
    inl child = lock_ports host port |> async.start_child |> async.let'
    trace Debug (fun () => "2") id
    async.sleep 240 |> async.do
    trace Debug (fun () => "3") id
    inl available_port = get_available_port (None |> optionm'.box) host port |> async.let'
    trace Debug (fun () => "4") id
    inl retries = wait_for_port_access (None |> optionm'.box) false host port |> async.let'
    trace Debug (fun () => "5") id
    child |> async.do
    trace Debug (fun () => "6") id
    (available_port, retries) |> return
|> async.new_async_unit
|> async.run_with_timeout 15000
|> function
    | Some (available_port, retries) =>
        available_port |> _assert_eq (port + 2)

        retries
        |> _assert_between
            if platform.is_windows () then 50i64 else 50
            if platform.is_windows () then 150 else 1200

        true
    | _ => false
|> _assert_eq true

00:00:00 d #1 1
00:00:00 d #2 _1
00:00:00 d #3 2
00:00:00 d #4 _2
00:00:00 d #5 _3
00:00:00 d #6 3
00:00:02 v #7 networking.test_port_open / { port = 5557; ex = System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.) }
00:00:02 d #8 4
00:00:03 v #9 networking.wait_for_port_access / { port = 5555; retry = 100; timeout = None; status = false }
00:00:04 d #10 _4
00:00:04 d #11 _5
00:00:06 v #12 networking.test_port_open / { port = 5555; ex = System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.) }
00:00:06 d #13 5
00:00:06 d #14 6
__assert_eq / actual: 5557 / expected: 5557
__assert_between / actual: 111L / expected: struct (50L, 150L)
__assert_eq / actual: true / expected: true


In [ ]:
//// test

inl lock_ports host port = async.new_async_unit fun () =>
    trace Debug (fun () => "_1") id
    inl listener1 = new_tcp_listener (ip_address_parse host) port |> use
    inl listener2 = new_tcp_listener (ip_address_parse host) (port + 1) |> use
    trace Debug (fun () => "_2") id
    listener1 |> listener_start
    listener2 |> listener_start
    trace Debug (fun () => "_3") id
    async.sleep 400 |> async.do
    trace Debug (fun () => "_4") id
    listener1 |> listener_stop
    listener2 |> listener_stop
    trace Debug (fun () => "_5") id

inl host = "127.0.0.1"
inl port = 5555

fun () =>
    trace Debug (fun () => "1") id
    inl child = lock_ports host port |> async.start_child |> async.let'
    trace Debug (fun () => "2") id
    async.sleep 240 |> async.do
    trace Debug (fun () => "3") id
    inl available_port = get_available_port (Some 60 |> optionm'.box) host port |> async.let'
    trace Debug (fun () => "4") id
    inl retries = wait_for_port_access (Some 60 |> optionm'.box) false host port |> async.let'
    trace Debug (fun () => "5") id
    child |> async.do
    trace Debug (fun () => "6") id
    (available_port, retries) |> return
|> async.new_async_unit
|> async.run_with_timeout 1500
|> function
    | Some (available_port, retries) =>
        available_port |> _assert_eq (port + 2)

        retries
        |> _assert_between
            (if platform.is_windows () then 2i64 else 1)
            (if platform.is_windows () then 10 else 120)

        true
    | _ => false
|> _assert_eq true

00:00:00 d #1 1
00:00:00 d #2 2
00:00:00 d #3 _1
00:00:00 d #4 _2
00:00:00 d #5 _3
00:00:00 d #6 3
00:00:00 v #7 async.run_with_timeout_async / { timeout = 60 }
00:00:00 d #8 4
00:00:00 d #9 _4
00:00:00 d #10 _5
00:00:00 v #11 async.run_with_timeout_async / { timeout = 60 }
00:00:00 d #12 5
00:00:00 d #13 6
__assert_eq / actual: 5557 / expected: 5557
__assert_between / actual: 4L / expected: struct (2L, 10L)
__assert_eq / actual: true / expected: true


## main

In [ ]:
inl main () =
    init_trace_state None
    $'let test_port_open x = !test_port_open x' : ()
    $'let test_port_open_timeout x = !test_port_open_timeout x' : ()
    $'let wait_for_port_access x = !wait_for_port_access x' : ()
    $'let get_available_port x = !get_available_port x' : ()